#**Project Group 5**
### *Linguistic: Your auto essay grading tool*

##**Load Library**

In [ ]:
 !pip install language_tool_python
 !pip install textstat
 !pip install textacy
 !pip install pyspellchecker
 !pip install Jamspell
 !pip install Symspellpy
 !pip install Textblob 
 !pip install gensim
 !pip install lexrank
 nltk.download('punkt')
 nltk.download('stopwords')
 nltk.download('averaged_perceptron_tagger')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 105 kB 28.5 MB/s 
     |████████████████████████████████| 2.0 MB 57.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 200 kB 16.9 MB/s 
     |████████████████████████████████| 132 kB 62.2 MB/s 
     |████████████████████████████████| 1.6 MB 64.2 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=74007 sha256=aa0a2abc58fb7972848b9c20a82780c864b85487049572502d5e3e8713ac4cd1
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████

NameError: ignored

In [ ]:
import re
import nltk
import spacy
import gensim
import string
import textacy
import textstat
import collections
import language_tool_python

import numpy as np
import pandas as pd
import gensim.corpora as corpora

from path import Path
from nltk import tokenize
from scipy.stats import entropy
from textacy import preprocessing
from spellchecker import SpellChecker
from lexrank import STOPWORDS, LexRank
from textstat.textstat import textstatistics

from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

##**Import Data**

In [ ]:
data = pd.read_csv('train.csv')
data.head()

##**Feature Engineering**

**1. Number of words without punctuations**



In [ ]:
data['number_of_words'] = [len(text.split()) for text in data['full_text']]
data['number_of_words'].head()

In [ ]:
data['words'] = [nltk.word_tokenize(x) for x in data['full_text']]
data['words'].head()

In [ ]:
# Count the number of words without punctuations
data['alter_words'] = [nltk.RegexpTokenizer(r'\w+').tokenize(x) for x in data['full_text']]
data['alter_words'].head(10)

**2. Frequency of stop words**

In [ ]:
from nltk.corpus import stopwords

data['stopwords'] = 0
i = 0
for w in data['words']:
    counter = 0
    for k in w:
        if k in stopwords.words('english'):
            counter += 1
    data['stopwords'][i] = counter
    i += 1
 

data['stopwords'].head()

In [ ]:
data['stopwords_frequency'] = data['stopwords'] / data['number_of_words']

**3. Average number of words in a sentence**

In [ ]:
from nltk.tokenize import sent_tokenize
data['sentence'] = 0
data['sentence'] = [nltk.sent_tokenize(x) for x in data['full_text']]

In [ ]:
data['sentence'] = [len(x) for x in data['sentence']]

In [ ]:
data['av_word_per_sen'] = data['number_of_words'] / data['sentence']

**4. Frequency of punctuations**

In [ ]:
import string
i = 0
data['punctuations'] = 0
for word in data['words']:
    count = len([c for c in word if c in list(string.punctuation)])
    data['punctuations'][i] = count
    i += 1

In [ ]:
data['word_punc'] = [len(x) for x in data['words']]
data['punctuations'] = data['punctuations'] / data['word_punc']

**5. Automated Readability Index**

In [ ]:
data['word_no_pun'] = [x.split() for x in data['full_text']]

In [ ]:
data['ARI'] = 0
i = 0
for word_lst in data['word_no_pun']:
    ttl_characters = 0
    ari = 0
    for word in word_lst:
        ttl_characters += len(word)
    ari = 4.71*(ttl_characters / data['number_of_words'][i]) + 0.5*(data['number_of_words'][i] / data['sentence'][i]) - 21.43
    data['ARI'][i] = ari
    i += 1

In [ ]:
data = data.drop(['stopwords','sentence','word_no_pun','word_punc'], axis=1)

**6. Frequency of wrongly spelt words / text accuracy**

In [ ]:
spell = SpellChecker()
data['freq_of_wrong_words'] = [len(spell.unknown(data['words'][i])) / len(data['full_text'][i].split()) for i in range(len(data))]
data['freq_of_wrong_words'].head()

**7. POS Tagging Related**

In [ ]:
# Apply POS tagging
# Stemming to reduce to base form before tagging
tokenized_text=data['words'].tolist()
num_of_verb=[]
num_of_adj=[]
num_of_adv=[]
num_of_pron = []
num_of_tran = []
num_of_distinct_adj=[]
num_of_distinct_adv=[]
num_of_noun=[]
for i in range(len(tokenized_text)):
    sample = tokenized_text[i]
    ps = PorterStemmer()
    sample_tokens=[ps.stem(word) for word in sample]
    sample_tokens
    
    # Set to be 1 for smoothing
    sample_verb=1
    sample_adj=1
    sample_adv=1
    sample_noun=1
    sample_pron=1
    sample_transition=1
    
    adj_list=[]
    adv_list=[]

    tag=nltk.pos_tag(sample_tokens)
    for j in range(len(tag)):
        if tag[j][1] == 'VB':
            sample_verb+=1
            continue
        if tag[j][1][:2] == 'JJ':
            sample_adj+=1
            adj_list.append(tag[j][0])
            continue
        if tag[j][1][:2] == 'RB':
            sample_adv+=1
            adv_list.append(tag[j][0])
            continue
        if tag[j][1] == 'NN' or tag[j][1]=='NNS':
            sample_noun+=1
            continue
        if tag[j][1][:3] == 'PRP':
            sample_pron += 1
        if tag[j][1] == 'CC':
            sample_transition += 1
        if j < len(tag) -2 and tag[j][1] == 'IN' and tag[j+1][1] == 'NN' and tag[j+2][1] == ',':
            sample_transition += 1      

    num_of_verb.append(sample_verb)
    num_of_adj.append(sample_adj)
    num_of_adv.append(sample_adv)
    num_of_pron.append(sample_pron)
    num_of_tran.append(sample_transition)
    num_of_noun.append(sample_noun)
    num_of_distinct_adj.append(len(np.unique(np.array(adj_list))))
    num_of_distinct_adv.append(len(np.unique(np.array(adv_list)))) 

In [ ]:
data['freq_of_verb']=num_of_verb/data['number_of_words']
data['freq_of_adj']=num_of_adj/data['number_of_words']
data['freq_of_adv']=num_of_adv/data['number_of_words']
data['freq_of_distinct_adj']=num_of_distinct_adj/data['number_of_words']
data['freq_of_distinct_adv']=num_of_distinct_adv/data['number_of_words']
data['freq_of_noun']=num_of_noun/data['number_of_words']
data['freq_of_transition']=num_of_tran/data['number_of_words']
data['freq_of_pronoun']=num_of_pron/data['number_of_words']
data.head()

In [ ]:
# Cross feature
data['noun_to_adj']=data['freq_of_adj']/data['freq_of_noun']
data['verb_to_adv']=data['freq_of_adv']/data['freq_of_verb']

data['phrase_diversity']=data['noun_to_adj']*data['verb_to_adv']
data.head()

In [ ]:
# Split into sentences
sentence_complexity=[]
for i in range(3911):
    sample=data['corrected_text'][i]
    sentences=nltk.sent_tokenize(sample)
    entropy_list=[]
    # For each sentence, find total number of each tag
    # Calculate entropy
    # Average all entropy to get avg sentence complexity for this text
    for sentence in sentences:
        ps = PorterStemmer()
        stemmed=[ps.stem(word) for word in sentence]
        tags=nltk.pos_tag(stemmed)

        tag_dic={}
        for tag in tags:
            if tag[1] in tag_dic.keys():
                tag_dic[tag[1]]+=1
            else:
                tag_dic[tag[1]]=1

        entropy_list.append(entropy(list(tag_dic.values())))
    
    sentence_complexity.append(sum(entropy_list) / len(entropy_list))

data['sentence_complexity']=sentence_complexity

**8. Sentiment Scores**

In [ ]:
nltk.download('vader_lexicon')
  
sia=SentimentIntensityAnalyzer()

data['sentiment_compound']=data['full_text'].apply(lambda x: sia.polarity_scores(x).get('compound'))
data['sentiment_positive']=data['full_text'].apply(lambda x: sia.polarity_scores(x).get('pos'))
data['sentiment_negative']=data['full_text'].apply(lambda x: sia.polarity_scores(x).get('neg'))

**9. Grammar and Spell Checking**

In [ ]:
# Count the number of grammar errors and replace with correct form
tool = language_tool_python.LanguageTool('en-US')

data['num_of_grammar_errors'] = data['full_text'].apply(lambda x: len(tool.check(x.replace('\n',''))))
data['corrected_text'] = data['full_text'].apply(lambda x: tool.correct(x.replace('\n','')))
data.head()

Short Forms (combined with grammar error)

In [ ]:
# Find short forms
collections = {'u':'you', 'b':'B'}

def num_of_short_form(text):
    num_of_error=0
    splitted_text=text.split()
    for word in splitted_text:
        if word in collections.keys():
            num_of_error += 1
    return num_of_error     

In [ ]:
data['num_of_short_forms'] = data['corrected_text'].apply(lambda x: num_of_short_form(x))
data['Incorrect_form_ratio'] = (data['num_of_grammar_errors'] + data['num_of_short_forms'])/ data['number_of_words']
data.head()

**11. Scores from textstat**

In [ ]:
# Flesch reading ease: score between 1 to 100, 100 means highest readability
data['flesch_reading_ease']=data['full_text'].apply(lambda x: textstat.flesch_reading_ease(preprocessing.normalize.whitespace(x)))

# Flesch kincaid grade: required education level in order to read the text
data['flesch_kincaid_grade']=data['full_text'].apply(lambda x: textstat.flesch_kincaid_grade(preprocessing.normalize.whitespace(x)))

# Dale-Chall readability score: higher score means harder to understand
data['dale_chall_readability_score']=data['full_text'].apply(lambda x: textstat.dale_chall_readability_score(preprocessing.normalize.whitespace(x)))

# Text_standard score: a score provided by the library based on the score of other measures
data['text_standard']=data['full_text'].apply(lambda x: textstat.text_standard(preprocessing.normalize.whitespace(x), float_output=True))

# McAlpine-EFLAW score: higher the score, higher to understand, should be aiming below 25
data['mcalpine_eflaw']=data['full_text'].apply(lambda x: 25 - (textstat.mcalpine_eflaw(preprocessing.normalize.whitespace(x))))

In [ ]:
data.head()

**12. Difficult Word Extraction**

In [ ]:
# Takes VERY LONG
# Difficult words are those with syllables >= 2 (can customize the setting)

def syllables_count(word):
    return textstatistics().syllable_count(word)

def break_sentences(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return list(doc.sents)

def difficult_words(text):  
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    # Find all words in the text
    words = []
    sentences = break_sentences(text)
    for sentence in sentences:
        words += [str(token) for token in sentence]
 
    diff_words_set = set()
     
    for word in words:
        syllable_count = syllables_count(word)
        if word not in nlp.Defaults.stop_words and syllable_count >= 2:
            diff_words_set.add(word)
 
    return len(diff_words_set)

In [ ]:
data['number_of_diff_words'] = data['full_text'].apply(lambda x: difficult_words(x))
data['freq_diff_words']=data['number_of_diff_words'] / data['number_of_words']

**13. Type-Token Ratio (TTR)**

In [ ]:
# Type-token ratio for lexical measure
# Larger means higher lexical variation
def ttr(sample):
    sample_words=sample.split()
    n_words = len(sample_words)

    # remove all punctuations
    for i in range(n_words):
        for c in string.punctuation:
            sample_words[i] = sample_words[i].replace(c,'')

    # remove empty words
    sample_words = list(filter(None, sample_words))
    n_words = len(sample)

    # count each word
    word_count = collections.Counter(sample_words)

    # get the sorted list of unique words
    unique_words = list(word_count.keys())
    unique_words.sort()

    n_unique = len(unique_words)
    ttr = len(word_count)/float(n_words)
    return ttr

In [ ]:
data['ttr']=data['corrected_text'].apply(lambda x: ttr(x))
data.head()

**14. Coherence Score**

In [ ]:
# Convert sentences to words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

stop_words = stopwords.words('english')
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Function for final calculation
def coherence_score(essay):
    # Text to sentence then to words
    sentences=nltk.tokenize.sent_tokenize(essay)
    sentences_df=pd.DataFrame(sentences, columns=['sentence'])
    
    # Remove punctuation
    sentences_df['sentence'] = sentences_df['sentence'].apply(lambda x: re.sub('[,\.!?]', '', x))
    # Convert the titles to lowercase
    sentences_df['sentence'] = sentences_df['sentence'].apply(lambda x: x.lower())
    
    sentences = sentences_df.sentence.values.tolist()
    words = list(sent_to_words(sentences))
    
    # Build bigram model
    bigram = gensim.models.Phrases(words, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    
    # Remove Stop Words
    data_words_nostops = remove_stopwords(words)
    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)
    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)
    # Create Corpus
    texts = data_lemmatized
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                        id2word=id2word,
                        num_topics=5, 
                        random_state=100,
                        chunksize=100,
                        passes=10,
                        per_word_topics=True)
    
    # Compute coherence score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    
    return coherence_lda

data['coherence_score']=data['corrected_text'].apply(lambda x: coherence_score(x))

**15. Lexrank**

In [ ]:
# Corrected text to documents for lexrank, no need to run again
sample_df=pd.DataFrame(data['corrected_text'])
for index, row in sample_df.iterrows():
    if index > len(sample_df):
       break
    else:
       f = open('Text Documents/'+str(index)+'.txt', 'w', encoding='utf-8')
       f.write(row[0])
       f.close()
       index+=1

FileNotFoundError: ignored

In [ ]:
documents = []
documents_dir = Path('Text Documents')

for file_path in documents_dir.files('*.txt'):
    with file_path.open(mode='rt', encoding='utf-8') as fp:
        documents.append(fp.readlines())
        
lxr = LexRank(documents, stopwords=STOPWORDS['en'])

In [ ]:
data['lexrank_avg_min_diff'] = data['corrected_text'].apply(lambda x: sum(lxr.rank_sentences(tokenize.sent_tokenize(x), threshold=None))/len(lxr.rank_sentences(tokenize.sent_tokenize(x),threshold=None))-min(lxr.rank_sentences(tokenize.sent_tokenize(x), threshold=None)))
data['lexrank_interquartile'] = data['corrected_text'].apply(lambda x: np.percentile(lxr.rank_sentences(tokenize.sent_tokenize(x), threshold=None),[25,75])[1]-np.percentile(lxr.rank_sentences(tokenize.sent_tokenize(x), threshold=None),[25,75])[0])